# Load the pretrained model

Archive Images (on Server!)

In [ ]:
tar -czvf /home/alex/all_imgs_11_10_23.tar.gz

Download archive from server and extract it

In [33]:
!scp alex@foodsnapai.com:/home/alex/all_imgs_11_10_23.tar.gz /home/alex/allImgs.tar.gz
!tar -xzvf /home/alex/allImgs.tar.gz -C /home/alex/allImgs_extracted/

all_imgs_11_10_23.tar.gz                      100% 5940MB  89.4MB/s   01:06    
./
./4ef321f9-062e019baa094e4b_20230814_164634.jpg
./ce526192-f3e20bac582fe364_20230623_104746.jpg
./243d5d40-1868262dc749d77f_IMG_20231031_124534_HDR.jpg
./dfabe24b-48147482503e4702_20230625_132207.jpg
./3b27d0fe-03a23fd9f14e868b_20230929_232128.jpg
./a1508df1-ce38b76981283a0c_IMG_20230922_150753.jpg
./6a3140a2-1ad416aef812cc25_20230713_065710.jpg
./4a400252-4f63423f38f48fe7_20230619_202905.jpg
./ff4d8195-61fd3bad0e74e033_IMG_20230825_153304_HDR.jpg
./f9dc3180-1fc3389d2794ffe0_IMG_20230623_190857.jpg
./50bf9a83-166c3ce764c5c3be_IMG_20230906_204253.jpg
./b282e43b-e47398b0b8957f64_16915957892996911567048667487206.jpg
./6310f159-9dfc1304c638331d_16929415783254036852923991469656.jpg
./65d474e4-28350a67ee3af5c7_20230815_072727.jpg
./a2702439-34e03152417acd9b_IMG_8311.jpeg
./3392d64b-75eefce8689d7102_IMG_20230802_203236.jpg
./9a6cd074-feda5858694dbb00_0919A58B-930D-4CEC-91E1-5EDA515E8F2A.jpeg
./635303c0-97306c53

Load annotations without Images

In [1]:
import os

# Get the value of the environment variable
labelstudio_token = os.getenv('LABELSTUDIO_TOKEN')
filename = "annotations"
!curl -X GET http://foodsnapai.com:8080/api/projects/1/export?exportType=CSV -H 'Authorization: Token {labelstudio_token}' --output {filename +".csv"} 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2890k  100 2890k    0     0   348k      0  0:00:08  0:00:08 --:--:--  707k


Convert annotations and images and preprocess them

In [9]:
%run 'convert_csv_to_mlflow_csv.py' {filename+".csv"} 

import pandas as pd
import os
mlflow_filename = filename + "_mlflow.csv"
# Read the CSV file into a pandas DataFrame
df = pd.read_csv(mlflow_filename)

# Count distinct values in the 'path' column
distinct_files = df['path'].unique()
num_distinct_files = len(distinct_files)

# Rename the file to include the distinct path values
base_name, extension = os.path.splitext(filename)
new_filename = f"annotations_{num_distinct_files}{extension}"
os.rename(filename + ".csv", new_filename)

print(f"Distinct files in dataset: {num_distinct_files}")
print(f"File renamed to: {new_filename}")
annotations_base_filename = new_filename

Distinct files in dataset: 1907
File renamed to: annotations_1907


In [36]:
# merge classes here
# curret merge string: not:Tomato,Cucumber,Bell-Pepper,Onion,Carrot,Banana,Egg,Scallion,Lemon,Potato,Zucchini,Garlic,Apple,Pumpkin,Ginger
%run 'convert_pascal_to_googlecsv.py' --merged_csv {annotations_base_filename +"_mlflow.csv"} 

Skipping automatic class merge.
['Tomato', 'Cucumber', 'Bell-Pepper', 'Onion', 'Carrot', 'Banana', 'Egg', 'Scallion', 'Lemon', 'Potato', 'Zucchini', 'Garlic', 'Apple', 'Pumpkin', 'Ginger']
Merging classes ['Orange', 'Balsamico', 'Canned-Tomato', 'Vinegar', 'Leek', 'Mushroom', 'Salad', 'Rice', 'Broccoli', 'Skyr', 'Lentils', 'Chickpeas', 'Mango', 'Cheese', 'Strawberry', 'Peas', 'Curd', 'Beans', 'Flour', 'Tofu', 'Chilli', 'Nuts', 'Lime', 'Corn', 'Milk', 'Spinach', 'Avocado', 'Pasta', 'Plantmilk', 'Eggplant', 'Butter', 'Cabagge', 'Oil', 'Yoghurt', 'Cauliflower', 'Soysauce'] to 'other'
Train-test-validation split applied to annotations1900_mlflow_shuffled.csv


Now rename the file paths in the file to their actual location (find & replace)

In [37]:
import csv
import os

# Constant path to prepend
constant_path = '/home/alex/allImgs_extracted_smaller/'

# CSV file path
csv_file_path = annotations_base_filename +"_mlflow_shuffled.csv"

# Open the CSV file and modify the paths in the second column
with open(csv_file_path, 'r+', newline='') as file:
    csv_reader = csv.reader(file)
    rows = list(csv_reader)

    for row in rows:
        if len(row) > 1:  # Ensure the row has at least two columns
            # Assuming the file path is in the second column (index 1)
            file_path = row[1]
            file_name = os.path.basename(file_path)  # Extract the filename from the original path
            row[1] = os.path.join(constant_path, file_name)  # Create the new path with the constant path and filename

    # Move the file pointer to the beginning
    file.seek(0)

    # Write the modified rows back to the CSV file
    csv_writer = csv.writer(file)
    csv_writer.writerows(rows)

print(f"CSV file '{csv_file_path}' modified with constant path replacing each file path in the second column.")



CSV file 'annotations1900_mlflow_shuffled.csv' modified with constant path replacing each file path in the second column.


In [67]:
!pwd
%run 'png_to_jpeg.py' {annotations_base_filename +"_mlflow_shuffled.csv"} 
#%run 'png_to_jpeg.py' annotations1650_mlflow_shuffled_n.csv execute this line after training session 

/home/alex/tflite_model_maker_wsl2
convert png image files to jpeg imgs


0it [00:00, ?it/s]

Converted 44
Ignored 12942 files because: Not a PNG file
Ignored 0 files because: Not found


In [39]:
# make images smaller (run only when new images have been loaded)

%run preproc_imgs.py /home/alex/allImgs_extracted /home/alex/allImgs_extracted_smaller 800
#%run preproc_imgs.py /mnt/z/annotated_individual/images/ /home/alex/individual_tiny_extracted_smaller 800

Resizing images:  15%|█▍        | 637/4266 [02:01<08:56,  6.77it/s]

Error processing e82677308afcf9cb_IMG_20230622_114747.jpg: cannot identify image file '/home/alex/allImgs_extracted/e82677308afcf9cb_IMG_20230622_114747.jpg'


Resizing images:  45%|████▍     | 1909/4266 [06:06<05:45,  6.81it/s]

Error processing 2ea95cbde7c6aca9_IMG_20230622_115029.jpg: cannot identify image file '/home/alex/allImgs_extracted/2ea95cbde7c6aca9_IMG_20230622_115029.jpg'


Resizing images: 100%|██████████| 4266/4266 [13:38<00:00,  5.21it/s]

7 images did not need resizing. Copied to /home/alex/allImgs_extracted_smaller/4059e4cdc4670770_20231012_131640.jpg
4257 did not need resizing. Copied to /home/alex/allImgs_extracted_smaller/4059e4cdc4670770_20231012_131640.jpg


In [40]:

%run normalize_csv.py {annotations_base_filename +"_mlflow_shuffled.csv"}  {annotations_base_filename +"_mlflow_shuffled_n.csv"} 

check the stats of the dataset splits

In [41]:
%run 'get_label_stats.py' {annotations_base_filename +"_mlflow_shuffled_n.csv"} 

There are 1907 different files in this set
There are 16 different classes in this set
Classes: other, Banana, Egg, Cucumber, Carrot, Scallion, Apple, Bell-Pepper, Zucchini, Ginger, Onion, Garlic, Lemon, Tomato, Pumpkin, Potato
other occurs in 1611 different files
Banana occurs in 379 different files
Egg occurs in 273 different files
Cucumber occurs in 450 different files
Carrot occurs in 381 different files
Scallion occurs in 367 different files
Apple occurs in 243 different files
Bell-Pepper occurs in 447 different files
Zucchini occurs in 284 different files
Ginger occurs in 192 different files
Onion occurs in 451 different files
Garlic occurs in 245 different files
Lemon occurs in 309 different files
Tomato occurs in 498 different files
Pumpkin occurs in 159 different files
Potato occurs in 255 different files

Class Distribution in Splits:

TRAIN:
other: 1289 files (24.6%)
Banana: 314 files (5.99%)
Egg: 220 files (4.2%)
Cucumber: 350 files (6.68%)
Carrot: 323 files (6.17%)
Scallion

Now you can start to load the model and train!

In [3]:
import numpy as np
import os

from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

# The current code uses Focal loss which has already weighted loss because of alpha and gamma
model_name = 'efficientdet-lite0' # EfficientDetLite1Spec must also be set accordingly!
custom_model_dir_name = 'model_'+"1907cw"#str(num_distinct_files)
epochs = 50
batch_size = 16
model_dir = 'models/'+model_name+'/'+custom_model_dir_name+'_e'+str(epochs)+'_b'+str(batch_size)
#spec = model_spec.get('efficientdet_lite1')
# check this url to check valid hparam values
# https://github.com/tensorflow/examples/blob/master/tensorflow_examples/lite/model_maker/third_party/efficientdet/hparams_config.py
spec = object_detector.EfficientDetLite0Spec( # change this also to correct model spec
    model_name = model_name,
    model_dir='/home/alex/checkpoints/',
    hparams='grad_checkpoint=true,strategy=gpus',
    epochs=epochs, batch_size=batch_size,
    steps_per_execution=1, moving_average_decay=0,
    var_freeze_expr='(efficientnet|fpn_cells|resample_p6)',
    tflite_max_detections=25
)


# Load the dataset

If you get an error here run the png to jpeg script again. Inference seems to fuck it up ^^

In [5]:
annotations_base_filename = 'annotations1900'
%run 'png_to_jpeg.py' {annotations_base_filename +"_mlflow_shuffled.csv"} 
import os
current_directory = os.getcwd()
print("Current Working Directory:", current_directory)
annotations_base_filename = "annotations1900"
file_path = '/home/alex/tflite_model_maker_wsl2/'+annotations_base_filename +"_mlflow_shuffled_n.csv"

if os.path.exists(file_path):
    print(f"The file '{file_path}' exists.")
train_data, validation_data, test_data = object_detector.DataLoader.from_csv(file_path)


convert png image files to jpeg imgs


0it [00:00, ?it/s]

Converted 44
Ignored 12942 files because: Not a PNG file
Ignored 0 files because: Not found
Current Working Directory: /home/alex/tflite_model_maker_wsl2
The file '/home/alex/tflite_model_maker_wsl2/annotations1900_mlflow_shuffled_n.csv' exists.


# Train the model

In [6]:
model = object_detector.create(train_data, model_spec=spec, train_whole_model=True, validation_data=validation_data)

ValueError: `class_weight` is only supported for Models with a single output.

### Write the model maker class list to a file!

In [6]:
import json
import os

if not os.path.exists(model_dir):
    os.makedirs(model_dir)

label_map = model.model_spec.config.label_map.as_dict()
# Writing the dictionary to a JSON file
with open(model_dir+'/label_map.json', 'w') as file:
    json.dump(label_map, file)

# run tensorboard to monitor

In [57]:
# !tensorboard --logdir=/home/alex/checkpoints_lite1_1700imgs/ --port=6006

Run on test data

In [7]:
model.evaluate(test_data)

12/12 [==============================] - 12s 615ms/step



{'AP': 0.15582107,
 'AP50': 0.23841226,
 'AP75': 0.1736558,
 'APs': 0.0,
 'APm': 0.08010899,
 'APl': 0.19573046,
 'ARmax1': 0.14294691,
 'ARmax10': 0.2737395,
 'ARmax100': 0.28900233,
 'ARs': 0.0,
 'ARm': 0.15355001,
 'ARl': 0.35994118,
 'AP_/Carrot': 0.0873449,
 'AP_/other': 0.27583206,
 'AP_/Banana': 0.2731409,
 'AP_/Egg': 0.35327166,
 'AP_/Cucumber': 0.15881357,
 'AP_/Scallion': 0.14768273,
 'AP_/Apple': 0.12550583,
 'AP_/Bell-Pepper': 0.11589285,
 'AP_/Zucchini': 0.02666068,
 'AP_/Ginger': 0.117014706,
 'AP_/Onion': 0.18206252,
 'AP_/Garlic': 0.09251632,
 'AP_/Lemon': 0.19825514,
 'AP_/Tomato': 0.14835635,
 'AP_/Pumpkin': 0.075721815,
 'AP_/Potato': 0.115064956}

### Export the model to tflite model

In [8]:
model.export(export_dir=model_dir)
print(f"exported to model to {model_dir}")

2023-11-26 21:59:25.648544: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2023-11-26 21:59:40.474441: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'resample_p7/PartitionedCall' has 1 outputs but the _output_shapes attribute specifies shapes for 3 outputs. Output shapes may be inaccurate.


Estimated count of arithmetic ops: 1.804 G  ops, equivalently 0.902 G  MACs


2023-11-26 21:59:44.616073: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2023-11-26 21:59:44.616122: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2023-11-26 21:59:44.620836: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpeitso6kg
2023-11-26 21:59:44.679677: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2023-11-26 21:59:44.679748: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmpeitso6kg
2023-11-26 21:59:44.892896: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2023-11-26 21:59:45.959769: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/tmpeitso6kg
2023-11-26 21:59:46.411365: I tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { serve }; Status: success: OK. Took 1790968

Estimated count of arithmetic ops: 1.804 G  ops, equivalently 0.902 G  MACs


fully_quantize: 0, inference_type: 6, input_inference_type: 3, output_inference_type: 0
2023-11-26 22:00:57.595403: I tensorflow/compiler/mlir/lite/flatbuffer_export.cc:1963] Estimated count of arithmetic ops: 1.804 G  ops, equivalently 0.902 G  MACs



exported to model to models/efficientdet-lite0/model_1907_e50_b16


TODO: Evaluate on TFLITE

In [ ]:
# TODO TEST
#model_dir='models/efficientdet-lite0/model_1907_e50_b16/model.tflite'
model.evaluate_tflite(model_dir+'/model.tflite', test_data)

Run inference script for visual output

In [9]:
# needs to be after training for some reason

%run 'png_to_jpeg.py' {annotations_base_filename +"_mlflow_shuffled_n.csv"}


%run do_inference.py --input_csv {annotations_base_filename +"_mlflow_shuffled_n.csv"}\
                       --model_url {model_dir}/model.tflite --output_dir output_inference1 --label_file {model_dir}/label_map.json

convert png image files to jpeg imgs


0it [00:00, ?it/s]

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Converted 0
Ignored 12986 files because: Not a PNG file
Ignored 0 files because: Not found
Predications will be saved to output_inference1


100%|██████████| 190/190 [04:32<00:00,  1.43s/it]
